In [1]:
import numpy as np
import pandas as pd

In [2]:
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Input, Dense, GRU, Embedding, CuDNNGRU
from keras.layers.recurrent import LSTM
from keras.layers import Dropout
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import KFold
import tensorflow.keras
from tensorflow.keras import metrics
from matplotlib import pyplot

ImportError: cannot import name 'Adam' from 'tensorflow.python.keras.optimizers' (C:\Users\engin\anaconda3\lib\site-packages\tensorflow\python\keras\optimizers.py)

In [ ]:
dataset = pd.read_csv('veriler.csv')

In [ ]:
dataset

In [ ]:
target = dataset['group'].values
data = dataset['tweet'].values

In [ ]:
target

In [ ]:
data

In [ ]:
target=np.array(target)


In [ ]:
num_words = 10000
tokenizer = Tokenizer(num_words=num_words)

In [ ]:
tokenizer.fit_on_texts(data)

In [ ]:
tokenizer.word_index

In [ ]:
data_tokens = tokenizer.texts_to_sequences(data)

In [ ]:
data_tokens

In [ ]:
num_tokens = [len(tokens) for tokens in data_tokens]
num_tokens = np.array(num_tokens)

In [ ]:
num_tokens[0]

In [ ]:
np.mean(num_tokens)

In [ ]:
np.max(num_tokens)

In [3]:
np.argmax(num_tokens)

NameError: name 'num_tokens' is not defined

In [4]:
max_tokens = np.mean(num_tokens) + 2*np.std(num_tokens)
max_tokens = int(max_tokens)

max_tokens

NameError: name 'num_tokens' is not defined

In [5]:
np.std(num_tokens)

NameError: name 'num_tokens' is not defined

In [6]:
np.sum(num_tokens < max_tokens) / len(num_tokens)

NameError: name 'num_tokens' is not defined

In [7]:
data_pad = pad_sequences(data_tokens, maxlen=max_tokens)

NameError: name 'pad_sequences' is not defined

In [8]:
np.array(data_tokens[1])

NameError: name 'data_tokens' is not defined

In [9]:
data_pad=np.array(data_pad)

NameError: name 'data_pad' is not defined

In [10]:
idx = tokenizer.word_index
inverse_map = dict(zip(idx.values(), idx.keys()))

NameError: name 'tokenizer' is not defined

In [28]:
def tokens_to_string(tokens):
    words = [inverse_map[token] for token in tokens if token!=0]
    text = ' '.join(words)
    return text

In [29]:
tokens_to_string(data_tokens[1])

'agree'

In [30]:
num2_words=len(tokenizer.word_index)

In [31]:
num2_words

8871

In [32]:
embedding_size=100

In [33]:
word2vec = {}
with open('glove.6B.100d.txt', encoding='UTF-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        vec = np.asarray(values[1:], dtype='float32')
        word2vec[word] = vec

FileNotFoundError: [Errno 2] No such file or directory: 'glove.6B.100d.txt'

In [ ]:
embedding_matrix = np.random.uniform(-1, 1, (num2_words+1, embedding_size))
for word, i in tokenizer.word_index.items():
    if i < num2_words:
        embedding_vector = word2vec.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

In [ ]:
embedding_matrix.shape

In [ ]:
input_var = Input(shape=(None,), name='input_var')

In [ ]:
kfold=KFold(n_splits=5, shuffle=True, random_state=5)
for train,test in kfold.split(data_pad,target):
    x_train,x_test,y_train,y_test=data_pad[train],data_pad[test],target[train],target[test]
    model = Sequential()
    embedding_size = 100
    model.add(Embedding(input_dim=num2_words+1,
                              output_dim=embedding_size,
                              weights=[embedding_matrix],
                              trainable=True,
                              name='embedding-layer'))
    model.add(LSTM(units=16, return_sequences=True))

    model.add(LSTM(units=8, return_sequences=True))
    model.add(LSTM(units=4, return_sequences=False))
    model.add(Dropout(0.43))

    model.add(Dense(4, activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
    model.summary()
    train=model.fit(x_train, y_train, epochs=50, batch_size=256)
    result = model.evaluate(x_test, y_test)
    pyplot.plot(train.history['accuracy'])
    pyplot.show()
    pyplot.plot(train.history['loss'])
    pyplot.show()
    